# 최종 예측 - 빅파이썬

### 필요 함수 import

In [1]:
import numpy as np
import pandas as pd

In [2]:
from lightgbm import LGBMRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import BaggingRegressor

### 데이터 불러오기

In [34]:
train = pd.read_csv("./Train.csv") # train set
test = pd.read_csv("./Predict.csv") # test set

In [4]:
submission = pd.read_csv("./제출데이터.csv") # 제출 파일

In [5]:
# 상품군별 모델
model = {i:BaggingRegressor(LGBMRegressor(objective='regression',num_leaves=20,
                              learning_rate=0.1, n_estimators=900,
                              max_bin = 150, bagging_fraction = 0.9,
                              bagging_freq = 5, feature_fraction = 0.3,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf = 6, min_sum_hessian_in_leaf = 11,random_state = 1111),random_state = 1111) for idx,i in enumerate(test["상품군"].unique())}

## 예측 함수 생성

In [6]:
# 성능 평가 함수 생성
def mape(t, y):
    return 100*(np.abs((y-t)/t).sum())/t.shape[0]

In [7]:
class BigPython_Model:
    def __init__(self, model):
        
        # 상품군별 모델
        self.models=model
        
        # 코로나 effect
        self.corona={'의류':1.192,'속옷':1,'이미용':0.996,'주방':1,'농수축':1.236,'가전':1.188,'생활용품':1.292,'가구':1.328,'잡화':0.996,'건강기능':1.62,'침구':1}
        
    def Split_Category(self, data, train=True): # 카테고리별로 나눔

        # 상품군별로 데이터 반환시킴
        for c,x in data.groupby(["상품군"]):
            del x["상품군"]
            
            if train: # 훈련시
                # 카테고리 이름, X, y 반환
                yield c, x.drop(["취급액"], axis=1), x["취급액"]
                
            else:  # 예측시
                # 카테고리 이름, X 반환
                yield c, x
    
    def Train(self, data,train=True): # MAPE 결과를 출력하는 함수
        # 상품군별 데이터를 받아옴
        for c, X, y in self.Split_Category(data.copy()):
            # 훈련
            self.models[c].fit(X,y)
    

    def Predict(self, data, train=False, corona = True):
        # 예측값을 넣을 배열
        self.predicted = np.zeros(data.shape[0])
        # 상품군별 데이터를 받아옴
        for c, X in self.Split_Category(data.copy(),train=train):
            if c=="무형":
                pass # 무형이면 예측으로 0이기 때문에 pass
            
            else:
                # 해당 상품군의 index를 받아옴
                predicted_idx = X.index
                if corona: # corona effect 가 있었을 시기 -> corona effect 적용
                    self.predicted[X.index] = self.models[c].predict(X)*self.corona[c] # 각 상품군 별로 모델 적용
                else:
                    self.predicted[X.index] = self.models[c].predict(X) # 각 상품군 별로 모델 적용
                
        return self.predicted
        
        

## Train

In [8]:
bpm1 = BigPython_Model(model)
bpm1.Train(train)

In [9]:
predicted = bpm1.Predict(train, train=False, corona=False)
mape(train["취급액"], predicted)

53.263273175828864

## Predict

In [36]:
result = bpm1.Predict(test, train=False, corona=True)

In [23]:
submission.loc[submission["상품군"]!="무형","취급액"]=result
submission["취급액"]=submission["취급액"].fillna(0)

In [44]:
submission["취급액"].to_csv("./최종예측파일_빅파이썬.csv", index=False)